In [4]:
# Loading the audio file 
import librosa
import numpy as np

def load_audio(file_path, sr=16000):
    y, sr = librosa.load(file_path, sr=sr, mono=True)
    return y, sr


In [5]:
# Extracting the features
def extract_features(file_path):
    y, sr = load_audio(file_path)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    mfccs_mean = np.mean(mfccs.T, axis=0)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    chroma_mean = np.mean(chroma.T, axis=0)
    mel = librosa.feature.melspectrogram(y=y, sr=sr)
    mel_mean = np.mean(mel.T, axis=0)
    features = np.hstack([mfccs_mean, chroma_mean, mel_mean])
    return features


In [6]:
# load the dataset here 

import os

def load_data(data_dir, categories):
    features = []
    labels = []
    for category in categories:
        category_dir = os.path.join(data_dir, category)
        for file_name in os.listdir(category_dir):
            file_path = os.path.join(category_dir, file_name)
            feature = extract_features(file_path)
            if feature is not None:
                features.append(feature)
                labels.append(category)
            else:
                print(f"Skipping file {file_path} due to feature extraction error")
    return np.array(features), np.array(labels)

data_dir = 'dataset/emotion'
categories = ['Angry', 'Calm',  'Disgust','Fearful','Happy', 'Neutral','Sad',  'Suprised']


X, y = load_data(data_dir, categories)


In [8]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_encoded = le.fit_transform(y)


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


In [6]:
# Hyperparameter and tuner usage for more accuracy of the data 


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras_tuner import HyperModel

class EmotionHyperModel(HyperModel):
    def build(self, hp):
        emotion_model = Sequential()
        emotion_model.add(Dense(
            units=hp.Int('units_input', min_value=32, max_value=512, step=32),
            activation='relu',
            input_shape=(180,)
        ))
        emotion_model.add(Dropout(rate=hp.Float('dropout_input', min_value=0.0, max_value=0.5, step=0.1)))
        for i in range(hp.Int('num_layers', 1, 3)):
            emotion_model.add(Dense(
                units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32),
                activation='relu'
            ))
            emotion_model.add(Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.0, max_value=0.5, step=0.1)))
        emotion_model.add(Dense(len(categories), activation='softmax'))
        emotion_model.compile(
            optimizer=hp.Choice('optimizer', ['adam', 'rmsprop']),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        return emotion_model


In [7]:
from kerastuner.tuners import RandomSearch

tuner = RandomSearch(
    EmotionHyperModel(),
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='tuner_dir',
    project_name='emotion_classification'
)


C:\Users\admin\AppData\Local\Temp\ipykernel_10476\1068680567.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


Reloading Tuner from tuner_dir\emotion_classification\tuner0.json


In [8]:
tuner.search(X_train, y_train, epochs=50, validation_data=(X_test, y_test))


In [10]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

emotion_model = tuner.hypermodel.build(best_hps)
emotion_model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

emotion_model.save('emotion_model_best.h5')


C:\Users\admin\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.1388 - loss: 48.9701 - val_accuracy: 0.0972 - val_loss: 15.8654
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1488 - loss: 15.8440 - val_accuracy: 0.1667 - val_loss: 12.4488
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1617 - loss: 14.8302 - val_accuracy: 0.2222 - val_loss: 7.4945
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2295 - loss: 10.7081 - val_accuracy: 0.2361 - val_loss: 13.6442
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1947 - loss: 11.7469 - val_accuracy: 0.3056 - val_loss: 11.7454
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1826 - loss: 11.7201 - val_accuracy: 0.1875 - val_loss: 9.8733
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2507 - loss: 7.9943 - val_accuracy: 0.2361 - val_loss: 9.0714
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2096 - loss: 8.0819 - val_accuracy: 0.2917 

In [11]:
loss, accuracy = emotion_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy}")


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5904 - loss: 1.2684 
Test Accuracy: 0.6111111044883728


# Language

In [12]:
# Creating the language model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras_tuner import HyperModel

#language contain categories - 'english' and 'non-english'
data_dir_lang="dataset/language"
language_categories = ['English',"Non-English"]
X_lang, y_lang = load_data(data_dir_lang, language_categories)
y_lang_encoded = LabelEncoder().fit_transform(y_lang)
X_train_lang, X_test_lang, y_train_lang, y_test_lang = train_test_split(X_lang, y_lang_encoded, test_size=0.2, random_state=42)

# Create the model
def lang_create_model(lang_input_shape, lang_num_classes):
    language_model = Sequential()
    language_model.add(Dense(128, input_shape=(lang_input_shape,), activation='relu'))
    language_model.add(Dropout(0.5))
    language_model.add(Dense(128, activation='relu'))
    language_model.add(Dropout(0.5))
    language_model.add(Dense(lang_num_classes, activation='softmax'))
    return language_model

# Get input shape and number of classes
lang_input_shape = X_train_lang.shape[1]
lang_num_classes = len(np.unique(y_train_lang))

# Create and train the language model
language_model = lang_create_model(lang_input_shape, lang_num_classes)
language_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
language_model.fit(X_train_lang, y_train_lang, epochs=100, batch_size=32, validation_data=(X_test_lang, y_test_lang))

model_save_path = 'Language_model.h5'


# Save the model using the new Keras format
language_model.save(model_save_path)



C:\Users\admin\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 75ms/step - accuracy: 0.5833 - loss: 23.7983 - val_accuracy: 0.8500 - val_loss: 6.2743
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7949 - loss: 11.3315 - val_accuracy: 0.8833 - val_loss: 3.5627
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8028 - loss: 7.5599 - val_accuracy: 0.9500 - val_loss: 0.8459
Epoch 4/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.7785 - loss: 6.1698 - val_accuracy: 0.9500 - val_loss: 0.8057
Epoch 5/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8334 - loss: 4.9218 - val_accuracy: 0.9667 - val_loss: 0.6971
Epoch 6/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8246 - loss: 3.8785 - val_accuracy: 0.9833 - val_loss: 0.2879
Epoch 7/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8860 - loss: 2.9650 - val_accuracy: 0.9833 - val_loss: 0.1072
Epoch 8/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8400 - loss: 4.0509 - val_accuracy: 0.9833 - val_los

# Gender Model

In [30]:
#  gender  contain categories - 'male' and 'female'
data_dir_gender="dataset/gender"
gender_categories = [ 'Female',"Male"]
X_gen, y_gen = load_data(data_dir_gender, gender_categories)
y_gen_encoded = LabelEncoder().fit_transform(y_gen)
X_train_gen, X_test_gen, y_train_gen, y_test_gen = train_test_split(X_gen, y_gen_encoded, test_size=0.2, random_state=42)

# Create the model
def gender_create_model(gender_input_shape, gender_num_classes):
    gender_model = Sequential()
    gender_model.add(Dense(128, input_shape=(gender_input_shape,), activation='relu'))
    gender_model.add(Dropout(0.5))
    gender_model.add(Dense(128, activation='relu'))
    gender_model.add(Dropout(0.5))
    gender_model.add(Dense(gender_num_classes, activation='softmax'))
    return gender_model

# Get input shape and number of classes
gender_input_shape = X_train_gen.shape[1]
gender_num_classes = len(np.unique(y_train_gen))

# Create and train the gender model
gender_model = gender_create_model(gender_input_shape, gender_num_classes)
gender_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
gender_model.fit(X_train_gen, y_train_gen, epochs=50, batch_size=32, validation_data=(X_test_gen, y_test_gen))

model_save_path = 'Gender_model.h5'


# Save the model using the new Keras format
gender_model.save(model_save_path)



Epoch 1/50


C:\Users\admin\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.5207 - loss: 29.8628 - val_accuracy: 0.3896 - val_loss: 20.8545
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5202 - loss: 25.0491 - val_accuracy: 0.4805 - val_loss: 5.4109
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5497 - loss: 17.9872 - val_accuracy: 0.7403 - val_loss: 1.9541
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6044 - loss: 11.4567 - val_accuracy: 0.7403 - val_loss: 1.9906
Epoch 5/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6263 - loss: 10.3186 - val_accuracy: 0.7662 - val_loss: 1.2102
Epoch 6/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6262 - loss: 11.4126 - val_accuracy: 0.7662 - val_loss: 1.1932
Epoch 7/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6215 - loss: 10.6446 - val_accuracy: 0.8701 - val_loss: 0.5351
Epoch 8/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7071 - loss: 6.2900 - val_accuracy: 0.9221 - val_l